In [17]:
import os
import platform
import psutil
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Inicializar o app
app = dash.Dash(__name__)

# Funções para coletar informações do sistema
def get_system_info():
    return {
        "Sistema Operacional": platform.system(),
        "Versão": platform.version(),
        "Arquitetura": platform.architecture()[0],
        "Processador": platform.processor(),
        "Nome do Host": platform.node()
    }

def get_hardware_info():
    mem = psutil.virtual_memory()
    disk = psutil.disk_usage('/')
    return {
        "CPU Uso (%)": psutil.cpu_percent(interval=None),
        "Memória Total (GB)": round(mem.total / 1e9, 2),
        "Memória Usada (GB)": round(mem.used / 1e9, 2),
        "Espaço Total Disco (GB)": round(disk.total / 1e9, 2),
        "Espaço Usado Disco (GB)": round(disk.used / 1e9, 2)
    }

def get_process_info():
    return [
        {
            "PID": proc.info["pid"],
            "Nome": proc.info["name"],
            "CPU (%)": proc.info["cpu_percent"],
            "Memória (%)": proc.info["memory_percent"],
        }
        for proc in psutil.process_iter(["pid", "name", "cpu_percent", "memory_percent"])
    ]

# Layout do Dashboard
app.layout = html.Div([
    html.H1("Dashboard do Sistema Operacional", style={'text-align': 'center'}),

    # Informações Globais
    html.Div([
        html.H3("Informações Globais"),
        html.Ul(id='system-info'),
        html.Ul(id='hardware-info')
    ], style={'margin': '20px'}),

    # Gráfico de uso da CPU
    dcc.Graph(id='cpu-usage', style={'margin': '20px'}),

    # Tabela de Processos
    html.Div([
        html.H3("Processos em Execução"),
        dcc.Dropdown(id='process-select', placeholder="Selecione um processo para detalhes"),
        html.Table(id='process-table', style={'width': '100%', 'border': '1px solid black'})
    ], style={'margin': '20px'}),

    # Detalhes de Processo
    html.Div([
        html.H3("Detalhes do Processo Selecionado"),
        html.Ul(id='process-details')
    ], style={'margin': '20px'}),

    # Intervalo de Atualização
    dcc.Interval(id='update-interval', interval=5000, n_intervals=0)
], style={
    'background-color': 'black',
    'color': 'white',
    'padding': '20px',
    'font-family': 'Arial, sans-serif'
})

# Callbacks
@app.callback(
    [Output('system-info', 'children'),
     Output('hardware-info', 'children'),
     Output('cpu-usage', 'figure'),
     Output('process-table', 'children'),
     Output('process-select', 'options')],
    [Input('update-interval', 'n_intervals')]
)
def update_dashboard(_):
    # Informações do sistema
    system_info = get_system_info()
    system_info_list = [html.Li(f"{key}: {value}") for key, value in system_info.items()]

    # Informações de hardware
    hardware_info = get_hardware_info()
    hardware_info_list = [html.Li(f"{key}: {value}") for key, value in hardware_info.items()]

    # Gráfico de uso de CPU
    figure = {
        'data': [{'x': list(range(10)), 'y': [psutil.cpu_percent() for _ in range(10)], 'type': 'line', 'name': 'CPU Usage'}],
        'layout': {'title': 'Uso da CPU (%)'}
    }

    # Processos em execução
    processes = get_process_info()
    table_header = html.Tr([html.Th(col) for col in ["PID", "Nome", "CPU (%)", "Memória (%)"]])
    table_rows = [
        html.Tr([html.Td(proc[col]) for col in ["PID", "Nome", "CPU (%)", "Memória (%)"]]) for proc in processes
    ]
    process_table = [table_header] + table_rows

    # Opções para o dropdown de seleção de processos
    process_options = [{'label': f"{proc['Nome']} (PID: {proc['PID']})", 'value': proc['PID']} for proc in processes]

    return system_info_list, hardware_info_list, figure, process_table, process_options

@app.callback(
    Output('process-details', 'children'),
    [Input('process-select', 'value')]
)
def show_process_details(selected_pid):
    if selected_pid is None:
        return [html.Li("Selecione um processo para exibir os detalhes.")]
    
    try:
        proc = psutil.Process(selected_pid)
        details = {
            "PID": proc.pid,
            "Nome": proc.name(),
            "Usuário": proc.username(),
            "CPU (%)": proc.cpu_percent(interval=0.1),
            "Memória (%)": proc.memory_percent(),
            "Status": proc.status(),
            "Tempo de Execução": proc.create_time()
        }
        return [html.Li(f"{key}: {value}") for key, value in details.items()]
    except psutil.NoSuchProcess:
        return [html.Li("O processo selecionado não está mais ativo.")]

# Rodar o servidor
if __name__ == '__main__':
    app.run_server(debug=True)


   ---------------------------------------- 0.0/222.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/222.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/222.5 kB ? eta -:--:--
   ----- --------------------------------- 30.7/222.5 kB 262.6 kB/s eta 0:00:01
   -------------- ------------------------ 81.9/222.5 kB 512.0 kB/s eta 0:00:01
   -------------------------------------- - 215.0/222.5 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 222.5/222.5 kB 1.1 MB/s eta 0:00:00
